In [5]:
# AI Studio平台默认安装了Paddle和PaddleNLP，并定期更新版本。 建议手动更新，保证使用最新版本
# 安装最新版PaddleNLP，首次更新完以后，重启后方能生效
!pip install --upgrade paddlenlp

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# 安装过程持续3分钟左右，与网络有关
!pip install paddle-pipelines==0.1.0

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 191.1/191.1 kB 1.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 9.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 MB 460.8 kB/s eta 0:00:0000:0100:03
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 409.6/409.6 kB 704.8 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 kB 749.1 kB/s eta 0:00:00 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 321.8/321.8 kB 779.3 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 62.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 711.0 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.5/40.5 kB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 100.3/100.3 kB 737.1 kB/s eta 0:00:00a 0:00:01
  Preparing meta

In [4]:
import os
# 导入paddle和pipelines相关的库
import paddle
from pipelines.document_stores import FAISSDocumentStore
from pipelines.utils import convert_files_to_dicts, fetch_archive_from_http, print_documents
from pipelines.nodes import ErnieRanker, DensePassageRetriever

In [5]:
doc_dir = "data/insurance"
city_data = "https://paddlenlp.bj.bcebos.com/applications/insurance.zip"
fetch_archive_from_http(url=city_data, output_dir=doc_dir)
dicts = convert_files_to_dicts(dir_path=doc_dir,
                                       split_paragraphs=True,
                                       split_answers=True,
                                       encoding='utf-8')

INFO - pipelines.utils.import_utils -  Fetching from https://paddlenlp.bj.bcebos.com/applications/insurance.zip to `data/insurance`
INFO - pipelines.utils.preprocessing -  Converting data/insurance/qa_pair.txt


In [6]:
# 删除原有的faiss文件
if(os.path.exists('faiss_document_store.db')):
    os.remove('faiss_document_store.db')
# 初始化FAISSDocumentStore，定义向量的维度和索引的类型
document_store = FAISSDocumentStore(embedding_dim=312,
                                            faiss_index_factory_str="Flat")
# 把文本数据写入到数据库
document_store.write_documents(dicts)

INFO - pipelines.document_stores.faiss -  document_cnt:0	embedding_cnt:0


Writing Documents:   0%|          | 0/3788 [00:00<?, ?it/s]

In [7]:
# 抽向量的batch_size
retriever_batch_size=16
# gpu的开关
use_gpu = True
# query文本的长度
max_seq_len_query = 64
# passage文本的长度
max_seq_len_passage = 384
retriever = DensePassageRetriever(
            document_store=document_store,
            query_embedding_model="rocketqa-zh-nano-query-encoder",
            passage_embedding_model="rocketqa-zh-nano-query-encoder",
            max_seq_len_query=max_seq_len_query,
            max_seq_len_passage=max_seq_len_passage,
            batch_size=retriever_batch_size,
            use_gpu=use_gpu,
            embed_title=False,
        )

INFO - pipelines.utils.common_utils -  Using devices: PLACE(GPU:0)
INFO - pipelines.utils.common_utils -  Number of GPUs: 1
[2022-11-14 10:41:08,618] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/rocketqa/rocketqa-zh-nano-query-encoder.pdparams and saved to /home/aistudio/.paddlenlp/models/rocketqa-zh-nano-query-encoder
[2022-11-14 10:41:08,621] [    INFO] - Downloading rocketqa-zh-nano-query-encoder.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/rocketqa/rocketqa-zh-nano-query-encoder.pdparams
100%|██████████| 68.3M/68.3M [00:09<00:00, 7.24MB/s]
W1114 10:41:18.664929   519 gpu_resources.cc:61] Please NOTE: device: 0, GPU Compute Capability: 7.0, Driver API Version: 11.2, Runtime API Version: 11.2
W1114 10:41:18.669476   519 gpu_resources.cc:91] device: 0, cuDNN Version: 8.2.
[2022-11-14 10:41:20,179] [    INFO] - Already cached /home/aistudio/.paddlenlp/models/rocketqa-zh-nano-query-encoder/rocketqa-zh-nano-query-encoder.pdparams
[2022

In [8]:
# 使用retriever抽取文本的向量，然后更新到faiss中
document_store.update_embeddings(retriever)

INFO - pipelines.document_stores.faiss -  Updating embeddings for 3776 docs...


Updating Embedding:   0%|          | 0/3776 [00:00<?, ? docs/s]

Create embeddings:   0%|          | 0/3776 [00:00<?, ? Docs/s]

In [9]:
# 将faiss的索引文件存储
index_name = "faiss_index"
document_store.save(index_name)

In [10]:
### Ranker
ranker = ErnieRanker(
        model_name_or_path="rocketqa-mini-cross-encoder",
        use_gpu=use_gpu)


INFO - pipelines.utils.common_utils -  Using devices: PLACE(GPU:0)
INFO - pipelines.utils.common_utils -  Number of GPUs: 1


Loading Parameters from:rocketqa-mini-cross-encoder


[2022-11-14 10:44:34,681] [    INFO] - Downloading https://paddlenlp.bj.bcebos.com/models/transformers/rocketqa/rocketqa-mini-cross-encoder.pdparams and saved to /home/aistudio/.paddlenlp/models/rocketqa-mini-cross-encoder
[2022-11-14 10:44:34,686] [    INFO] - Downloading rocketqa-mini-cross-encoder.pdparams from https://paddlenlp.bj.bcebos.com/models/transformers/rocketqa/rocketqa-mini-cross-encoder.pdparams
100%|██████████| 103M/103M [00:22<00:00, 4.81MB/s] 
[2022-11-14 10:44:57,644] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.tokenizer.ErnieTokenizer'> to load 'rocketqa-mini-cross-encoder'.
[2022-11-14 10:44:57,648] [    INFO] - Downloading https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_mini_zh_vocab.txt and saved to /home/aistudio/.paddlenlp/models/rocketqa-mini-cross-encoder
[2022-11-14 10:44:57,651] [    INFO] - Downloading ernie_3.0_mini_zh_vocab.txt from https://bj.bcebos.com/paddlenlp/models/transformers/ernie_3.0/ernie_3.0_mini_zh_

In [11]:
### Pipeline
from pipelines.pipelines import SemanticSearchPipeline
pipe = SemanticSearchPipeline(retriever, ranker)

In [12]:
pipeline_params = {"Retriever": {"top_k": 50}, "Ranker": {"top_k": 3}}

In [13]:
prediction = pipe.run(query="企业如何办理养老保险", params=pipeline_params)

In [14]:
print_documents(prediction, print_name=False, print_meta=True)


Query: 企业如何办理养老保险

{   'content': '如何办理企业养老保险',
    'meta': {   'answer': '企业养老保险一般是交由企业办理，个人需要准备好相关的文件即可。个人在参加企业养老保险的时候，需填报《参加企业基本养老保险人员基本情况表》，并提供以下证件和主要资料：1、身份证件及复印件；2、户口簿及复印件；3、以个人身份参保前原为职工身份的本人档案材料；4、曾在其他统筹地区参保的，重新登记应提供原参保所在地社保机构开具的《基本养老保险关系转移表》；5、与单位解除劳动关系的，应提供相关证明；6、省社保机构规定的其他证件资料。企业缴费以职工工资总额为基数，缴费比例为20%；职工个人缴费以本人全部工资收入为基数，月缴费工资超过全省上一年度职工平均工资300%以上的部分不计入，低于60%的按60%计算。职工个人应当缴纳的养老保险费，由所在单位从其工资中代扣代缴。',
                'name': 'qa_pair.txt',
                'vector_id': '3715'}}

{   'content': '企业养老每个月交一次，需怎样办理？',
    'meta': {   'answer': '办理社会保险登记对于首次参加养老保险的企业，需携带《营业执照》、批准成立的文件或其它执业证件、《组织机构统一代码证》、近期《职工工资发放表》、单位法定代表人身份证复印件、存档机构的存档证明，到养老保险科办理登记手续。新参保单位自办理社会保险登记之日起30日内，持《市社会保险登记表》和《社会保险登记证》复印件及相关证件资料，到当*区的*税*办理社会保险缴费登记手续，并到商业银行开缴费账户。企业申报养老保险程序如果企业需按月申报养老保险，并按以下要求到养老保险科办理手续：1、职工有增减变动时，需填写《建立表》、《恢复表》、《中断表》，并携带《职工养老保险手册》、转移单等相关资料；2、职工缴费工作变动时，要填写《职工缴费工资表》；3、企业填写《月申报表》，业务员核定盖章，由企业持《申报表》到桥*区*税*缴费。企业退休待遇审批程序公司职工在达到法定退休年龄时可以办理退休手续。企业携带拟定退休人员档案、《养老保险手册》（退休职工本人在《养老保险手册》上逐年签字）、身份证复印件，到劳动*认定、审核

上面是输出的结果，可以看到第一条是正确的答案，说明搜索得到了正确的答案了

# 7.更新 & 删除数据



+ 更新数据 ：对于数据更新，可以直接把`data/insurance`目录下的文本文件，替换成您所需要的文本，然后重新运行整个项目，就可以重新构建索引，构建您的定制化FAQ智能问答系统。

+ 删除数据：对于删除数据，可以直接把`data/insurance`目录下的文本文件，删除相应的文本，然后重新运行整个项目，就可以重新构建索引，构建您的定制化FAQ智能问答系统。


整个构建流程很快， Enjoy it！

- **如果对您有帮助，欢迎star收藏一下，不易走丢哦~链接指路：**[https://github.com/PaddlePaddle/PaddleNLP](https://github.com/PaddlePaddle/PaddleNLP)

<img src="https://ai-studio-static-online.cdn.bcebos.com/7f287240b16f4c4081d0b57ca808dce62769b13369434c64b9a26cfc48b6a978" width="70%" height="70%"> <br />


- **加入微信交流群，一起学习吧**

欢迎扫码填写基础问卷后，加入PaddleNLP技术交流群（微信），获得超多福利：

（1）领取官方团队准备的10GB NLP学习大礼包

（2）获取近期课程链接，及时跟进最新发布功能

（3）与众多NLPer、PaddleNLP工程师交流最新技术

<img src="https://ai-studio-static-online.cdn.bcebos.com/f19aa7fd3cb44638900c8c723aa4c51d38ad6557e4764408aa8da1d435436e06" width="200px" height="200px" >